<a href="https://colab.research.google.com/github/eloveman/Homework/blob/main/HW_LinearModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd

car_data_raw = pd.read_csv("https://cdn.c18l.org/vehicles_lab.csv")

Cleaning the data.

In [21]:

selected_columns = [
    'price', #variables we are predicting
    'year', #older cars should cost less so I want to see if year preticts price
    'manufacturer', #a porsh costs more than a honda
    'model', #there are more expensive and less expensive models
    'condition', #good condition cars should cost more
    'odometer' #cars with less miles should cost more
]

df = car_data_raw[selected_columns]





In [22]:
df= df.dropna()

In [23]:
df.head

<bound method NDFrame.head of         price    year manufacturer                        model  condition  \
0       15000  2013.0         ford                    f-150 xlt  excellent   
1       27990  2012.0          gmc  sierra 2500 hd extended cab       good   
2       34590  2016.0    chevrolet        silverado 1500 double       good   
3       35000  2019.0       toyota                       tacoma  excellent   
4       29990  2016.0    chevrolet        colorado extended cab       good   
...       ...     ...          ...                          ...        ...   
192650  39990  2017.0     infiniti        qx80 sport utility 4d       good   
192652  32990  2016.0     infiniti        qx80 sport utility 4d       good   
192653  33590  2018.0        lexus              gs 350 sedan 4d       good   
192654  23590  2019.0       nissan            maxima s sedan 4d       good   
192655  28990  2018.0        lexus              es 350 sedan 4d       good   

        odometer  
0       128000

In [24]:
#scaling the data so the columns (year and odometer have a mean of 0 and a standard deviation of 1)

from sklearn.preprocessing import StandardScaler
numeric = ['year', 'odometer']
scaler = StandardScaler()

#saving this transformed data to the name df
df.loc[:, numeric] = scaler.fit_transform(df[numeric])


In [25]:
data_types = df.dtypes

# Get a list of columns with data type 'object' or 'category'
categorical_columns = data_types[data_types == 'object'].index.tolist()

In [26]:
categorical_columns

['manufacturer', 'model', 'condition']

In [27]:
#Getting rid of the outliers by using the IQR method

Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

df


<ipython-input-27-41bb0ef94ffb>:3: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1 = df.quantile(0.25)
<ipython-input-27-41bb0ef94ffb>:4: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3 = df.quantile(0.75)
<ipython-input-27-41bb0ef94ffb>:6: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


,price,year,manufacturer,model,condition,odometer
0,15000,0.345049,ford,f-150 xlt,excellent,0.079395
1,27990,0.239983,gmc,sierra 2500 hd extended cab,good,-0.192321
2,34590,0.660246,chevrolet,silverado 1500 double,good,-0.371911
3,35000,0.975443,toyota,tacoma,excellent,-0.310053
4,29990,0.660246,chevrolet,colorado extended cab,good,-0.427795
...,...,...,...,...,...,...
192650,39990,0.765312,infiniti,qx80 sport utility 4d,good,-0.316174
192652,32990,0.660246,infiniti,qx80 sport utility 4d,good,-0.252268
192653,33590,0.870377,lexus,gs 350 sedan 4d,good,-0.365886
192654,23590,0.975443,nissan,maxima s sedan 4d,good,-0.359417


Engineered variables


In [28]:
#manufacturer and model are linked so I wanted to combine the two variables also if two manufacters have the same name for a model this variable
#would be able to distingish the two

df['combined_var'] = df['manufacturer'] + df['model']


<ipython-input-28-5b5cf981cb16>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_var'] = df['manufacturer'] + df['model']


In [29]:
#Making a variable to only get good or excellent condition cars, most people want a safe car and dont want a project car

df['good_or_excellent'] = (df['condition'].isin(['good', 'excellent'])).astype(int)


<ipython-input-29-136f1ca929ad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['good_or_excellent'] = (df['condition'].isin(['good', 'excellent'])).astype(int)


In [30]:
import pandas as pd

df['price_bin'] = pd.cut(df['price'], bins=range(0, int(df['price'].max()) + 10000, 10000),
                         labels=[f'${i}-{i+9999}' for i in range(0, int(df['price'].max()), 10000)])


<ipython-input-30-1fe863121036>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_bin'] = pd.cut(df['price'], bins=range(0, int(df['price'].max()) + 10000, 10000),


In [31]:
from sklearn.model_selection import train_test_split

# Assuming 'X' are the features and 'y' is the target variable (price_bin)
x = df.drop('price_bin', axis=1)  # Features
y = df['price_bin']  # Target variable

 #Spliting the dataset into training and testing samples at an 80:20 ratio

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [32]:
df = pd.get_dummies(df, columns=['manufacturer', 'model', 'condition'])
#converting the categorical variables

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# using multinomial
model = LogisticRegression(multi_class='multinomial', max_iter=1000)



In [35]:
from sklearn.linear_model import LinearRegression

lin_mod = LinearRegression().fit(
    df.loc[:, df.columns!='price'],  # Features (X)
    df['price']  # Target variable (y)
)



lin_mod.coef_

ValueError: ignored

In [ ]:
#checking with accurancy score

accuracy_score(y_train, y_test)

accuracy_score